# HRV-Stress_n-back-test

## read csv

In [1]:
import os
import os.path as osp
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
path_data = 'data/individual'

In [3]:
subject_list = os.listdir(path_data)

In [4]:
def get_mrr(rr):
    '''rr: distance between peaks in ms'''
    return np.mean(rr)
def get_mhr(rr):
    return np.mean(60000/rr)
def get_sdrr(rr, mrr):
    num = np.sum([np.math.pow(x,2) for x in rr-mrr])
    return np.sqrt(num/(np.size(rr)-1))
def get_sdhr(rr, mhr):
    num = np.sum([np.math.pow(x,2) for x in (60000/rr)-mhr])
    return np.sqrt(num/(np.size(rr)-1))
def get_cvrr(sdrr, mrr):
    return sdrr*100/mrr
def get_rmssd(rr):
    num = np.sum([np.math.pow(x,2) for x in np.diff(rr)])
    return np.sqrt(num/(np.size(rr)-1))
def get_prr20(rr):
    arr = np.abs(np.diff(rr))
    count = 0
    for arr_i in arr:
        if arr_i > 20:
            count += 1 
    return count*100 / (np.size(rr)-1)
def get_prr50(rr):
    arr = np.abs(np.diff(rr))
    count = 0
    for arr_i in arr:
        if arr_i > 50:
            count += 1 
    return count*100 / (np.size(rr)-1)

In [5]:
scaler = MinMaxScaler()
hrv_windows = [60000, 45000, 30000, 15000]

for subject_name in subject_list:
    # read nback
    df = pd.read_csv(osp.join(path_data,subject_name,'nback.csv'))
    nback_original_columns = df.columns
    
    # read peaks
    peaks = pd.read_csv(osp.join(path_data,subject_name,'peaks.csv')).to_numpy()
    
    for hrv_window in hrv_windows:
        mrr = []; mhr = []; sdrr = []; sdhr = []; cvrr = []; rmssd = []; prr20 = []; prr50 = [];
        for time_stamp in df['time_stamp']:
            peaks_slice = peaks[(peaks > time_stamp - hrv_window) * (peaks <= time_stamp)]
            rr = np.diff(peaks_slice.flatten())
            mrr.append(get_mrr(rr))
            mhr.append(get_mhr(rr))
            sdrr.append(get_sdrr(rr, mrr[-1]))
            sdhr.append(get_sdhr(rr, mhr[-1]))
            cvrr.append(get_cvrr(sdrr[-1], mrr[-1]))
            rmssd.append(get_rmssd(rr))
            prr20.append(get_prr20(rr))
            prr50.append(get_prr50(rr))
        df['mrr_'+str(hrv_window)[:2]+'s'] = mrr
        df['mhr_'+str(hrv_window)[:2]+'s'] = mhr
        df['sdrr_'+str(hrv_window)[:2]+'s'] = sdrr
        df['sdhr_'+str(hrv_window)[:2]+'s'] = sdhr
        df['cvrr_'+str(hrv_window)[:2]+'s'] = cvrr
        df['rmsd_'+str(hrv_window)[:2]+'s'] = rmssd
        df['prr20_'+str(hrv_window)[:2]+'s'] = prr20
        df['prr50_'+str(hrv_window)[:2]+'s'] = prr50
    
    # reaction_time
    reaction_time = [0]
    idx = 1
    while True:
        try:
            reaction_time.append(df['time_stamp'][idx]-df['time_stamp'][idx-1])
            idx += 1
        except:
            break
    df['reaction_time'] = reaction_time
        
    # drop initial nback (value == 0)
    df.drop(df[df['value'] == 0].index, inplace = True)
    df.reset_index(inplace=True,drop=True)

    # save
    df.to_csv(osp.join(path_data,subject_name,'reaction_time_raw.csv'), index=False)
    
    # penalty
    col_names = ['reaction_time']
    for penalty in np.arange(1.1,2.1,0.1):
        col_name = 'reaction_time_penalized_'+str(penalty)[0:3]
        col_names.append(col_name)
        df[col_name] = df['reaction_time']
        df.loc[df[df['value']==-1].index,col_name] = df['reaction_time'][df[df['value']==-1].index] * penalty
    
    # remove reaction_time outlier 
    lb = df['reaction_time'].quantile(.05)
    ub = df['reaction_time'].quantile(.95)

    inliers =df['reaction_time'].between(lb, ub)
    index_to_drop = df[~inliers].index
    df_inliers = df.drop(index_to_drop, inplace=False) # copy to make inliers only df and original
    df_inliers.reset_index(inplace=True,drop=True)
    
    # true only and penalized
    df_true = df.drop(df[df['value'] == -1].index, inplace = False)
    df_true.drop(columns=col_names[1:],inplace=True)
    df_true.reset_index(inplace=True,drop=True)
    
    df_penalized = df
    
    df_inliers_true = df_inliers.drop(df_inliers[df_inliers['value'] == -1].index, inplace = False)
    df_inliers_true.drop(columns=col_names[1:],inplace=True)
    df_inliers_true.reset_index(inplace=True,drop=True)
    
    df_inliers_penalized = df_inliers

    # save
    df_true.to_csv(osp.join(path_data,subject_name,'reaction_time_true.csv'), index=False)
    df_penalized.to_csv(osp.join(path_data,subject_name,'reaction_time_penalized.csv'), index=False)
    df_inliers_true.to_csv(osp.join(path_data,subject_name,'reaction_time_true_inliers.csv'), index=False)
    df_inliers_penalized.to_csv(osp.join(path_data,subject_name,'reaction_time_penalized_inliers.csv'), index=False)
    
    # minmax scaler
    for col_name in df_penalized.columns:
        if col_name not in nback_original_columns:
            df_penalized[col_name] = scaler.fit_transform(df_penalized[col_name].values.reshape(-1, 1))
    for col_name in df_inliers_penalized.columns:
        if col_name not in nback_original_columns:
            df_inliers_penalized[col_name] = scaler.fit_transform(df_inliers_penalized[col_name].values.reshape(-1, 1))
    for col_name in df_true.columns:
        if col_name not in nback_original_columns:
            df_true[col_name] = scaler.fit_transform(df_true[col_name].values.reshape(-1, 1))
    for col_name in df_inliers_true.columns:
        if col_name not in nback_original_columns:
            df_inliers_true[col_name] = scaler.fit_transform(df_inliers_true[col_name].values.reshape(-1, 1))

    # save
    df_true.to_csv(osp.join(path_data,subject_name,'reaction_time_true_minmax.csv'), index=False)
    df_penalized.to_csv(osp.join(path_data,subject_name,'reaction_time_penalized_minmax.csv'), index=False)
    df_inliers_true.to_csv(osp.join(path_data,subject_name,'reaction_time_true_inliers_minmax.csv'), index=False)
    df_inliers_penalized.to_csv(osp.join(path_data,subject_name,'reaction_time_penalized_inliers_minmax.csv'), index=False)

In [6]:
df.head()

,time_stamp,value,flag,mrr_60s,mhr_60s,sdrr_60s,sdhr_60s,cvrr_60s,rmsd_60s,prr20_60s,...,reaction_time_penalized_1.1,reaction_time_penalized_1.2,reaction_time_penalized_1.3,reaction_time_penalized_1.4,reaction_time_penalized_1.5,reaction_time_penalized_1.6,reaction_time_penalized_1.7,reaction_time_penalized_1.8,reaction_time_penalized_1.9,reaction_time_penalized_2.0
0,1572545903015,1,11,0.490955,0.491431,0.723507,0.750360,0.752415,0.762361,0.861681,...,0.121495,0.111186,0.102489,0.095055,0.088626,0.083011,0.078066,0.073677,0.069755,0.066229
1,1572545907711,1,11,0.535815,0.464542,0.775860,0.784800,0.786992,0.769867,0.749900,...,0.594905,0.544426,0.501844,0.465440,0.433960,0.406469,0.382253,0.360761,0.341557,0.324293
2,1572545908912,1,11,0.552501,0.453036,0.786679,0.793359,0.791274,0.769643,0.715301,...,0.137336,0.125683,0.115853,0.107449,0.100181,0.093835,0.088245,0.083283,0.078850,0.074864
3,1572545910146,-1,11,0.528453,0.487599,0.821205,0.843856,0.833605,0.791347,0.680702,...,0.157812,0.159206,0.160383,0.161388,0.162258,0.163017,0.163686,0.164280,0.164810,0.165287
4,1572545911609,1,11,0.527264,0.492374,0.832356,0.854882,0.844862,0.803551,0.652150,...,0.171637,0.157074,0.144788,0.134285,0.125203,0.117271,0.110285,0.104084,0.098543,0.093563


## Check NaN and Inf

In [7]:
for subject_name in subject_list:
    # read
    df = pd.read_csv(osp.join(path_data,subject_name,'reaction_time_raw.csv'))
    for col in df.columns:
        if df[col].isnull().values.any():
            print(subject_name, col)

In [8]:
for subject_name in subject_list:
    # read
    df = pd.read_csv(osp.join(path_data,subject_name,'reaction_time_raw.csv'))
    for col in df.columns:
        if not df.index[np.isinf(df[col])].empty:
            print(subject_name, col)

## Elbow Method

In [9]:
from sklearn.cluster import KMeans

In [10]:
from kneed import KneeLocator

def kneed_elbow(model, x, fit_num=1, cluster_nums=range(2,10)):
    distortions = []
    for cluster_num in cluster_nums:
        model.n_clusters=cluster_num
        inertia = 0
        for _ in range(fit_num):
            model.fit(x)
            inertia += model.inertia_
        distortions.append(inertia/fit_num)
    return KneeLocator(cluster_nums, distortions, S=1.0, curve="convex", direction="decreasing")

In [11]:
# elbow (based on true only, with outliers drop, and no penalty)
elbow_list = []
kneedle_list = []
for subject_name in subject_list:
    # read
    path_input_file = osp.join(path_data,subject_name,'reaction_time_true_inliers_minmax.csv')
    df = pd.read_csv(path_input_file)

    # Instantiate the clustering model and visualizer
    cluster = KMeans()
    kneedle = kneed_elbow(cluster, df['reaction_time'].values.reshape(-1, 1))
    elbow_list.append(kneedle.knee) 
    kneedle_list.append(kneedle)
elbow_num = sum(elbow_list)//len(elbow_list)

# result
# elbow_num = 4
print(elbow_num)

4


In [12]:
# sample plot
# kneedle_list[0].plot_knee(); kneedle_list[0].plot_knee_normalized()

## clustering k-means (to four groups, as elbow)

In [13]:
def sorted_cluster(x, model=None):
    if model == None:
        model = KMeans()
    model = sorted_cluster_centers_(model, x)
    model = sorted_labels_(model, x)
    return model

def sorted_cluster_centers_(model, x):
    model.fit(x)
    new_centroids = []
    magnitude = []
    for center in model.cluster_centers_:
        magnitude.append(np.sqrt(center.dot(center)))
    idx_argsort = np.argsort(magnitude)
    model.cluster_centers_ = model.cluster_centers_[idx_argsort]
    return model

def sorted_labels_(sorted_model, x):
    sorted_model.labels_ = sorted_model.predict(x)
    return sorted_model

In [14]:
file_names = [
    'reaction_time_true_minmax.csv', 
    'reaction_time_penalized_minmax.csv', 
    'reaction_time_true_inliers_minmax.csv', 
    'reaction_time_penalized_inliers_minmax.csv'
]

cluster = KMeans(n_clusters=elbow_num, random_state=0)
for subject_name in subject_list:
    for file_name in file_names:
        # rea
        df = pd.read_csv(osp.join(path_data,subject_name,file_name))
        for idx,col_name in enumerate(df.columns):
            if 'reaction_time' in col_name:
                cluster = sorted_cluster(df[col_name].values.reshape(-1,1), cluster)
                df[col_name] = cluster.labels_    
        df.to_csv(osp.join(path_data,subject_name,'kmeans_'+file_name), index=False)

In [15]:
df.head()

,time_stamp,value,flag,mrr_60s,mhr_60s,sdrr_60s,sdhr_60s,cvrr_60s,rmsd_60s,prr20_60s,...,reaction_time_penalized_1.1,reaction_time_penalized_1.2,reaction_time_penalized_1.3,reaction_time_penalized_1.4,reaction_time_penalized_1.5,reaction_time_penalized_1.6,reaction_time_penalized_1.7,reaction_time_penalized_1.8,reaction_time_penalized_1.9,reaction_time_penalized_2.0
0,1572545903015,1,11,0.490955,0.491431,0.723507,0.750360,0.752415,0.762361,0.861681,...,1,1,1,1,1,1,1,1,1,1
1,1572545908912,1,11,0.552501,0.453036,0.786679,0.793359,0.791274,0.769643,0.715301,...,2,1,1,1,2,2,2,1,1,1
2,1572545910146,-1,11,0.528453,0.487599,0.821205,0.843856,0.833605,0.791347,0.680702,...,2,2,2,2,2,3,3,2,2,2
3,1572545911609,1,11,0.527264,0.492374,0.832356,0.854882,0.844862,0.803551,0.652150,...,2,2,2,2,2,2,2,1,1,1
4,1572545913345,-1,11,0.544158,0.481837,0.848074,0.866360,0.853463,0.799475,0.611505,...,3,3,3,3,3,3,3,3,3,3


## split kde (to three groups, max)

In [16]:
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.signal import argrelextrema

def split_kde(x, model=None, start_end=None, n_groups=None, lower_bound=None, upper_bound=None, max_iter=100):
    '''
    x: data
    model: sklearn.neighbors.KernelDensity
    start_end: (optional) linspace start and end point
    n_groups: (optional) specify number of groups after splitted
    lower_bound: (optional) if n_groups, lower bound for binary search
    upper_bound: (optional) if n_groups, upper bound for binary search
    max_iter: maximum number of iteration
    '''
    if start_end == None:
        start = min(x)
        end = max(x)
    else:
        start, end = (start_end)
    
    if model == None:
        model = KernelDensity(kernel='gaussian', bandwidth=(end-start)/100)
    
    model.s_ = np.linspace(start,end)
    model.fit(x)
    model.e_ = model.score_samples(model.s_.reshape(-1,1))
    mi = argrelextrema(model.e_, np.less)[0]
    model.mi_ = mi
    if n_groups:
        # binary_search
        lower_bound = 0
        iteration = 0
        while len(mi) != (n_groups-1):
            if len(mi) > (n_groups-1):
                # increase kernel
                lower_bound = model.bandwidth
                if upper_bound:
                    model.bandwidth = lower_bound + (upper_bound-lower_bound) / 2
                else:
                    model.bandwidth = lower_bound * 2
            else:
                # decrease kernel
                upper_bound = model.bandwidth
                model.bandwidth = lower_bound + (upper_bound-lower_bound) / 2
            model.fit(x)
            model.e_ = model.score_samples(model.s_.reshape(-1,1))
            mi = argrelextrema(model.e_, np.less)[0]
            iteration += 1
            if iteration == max_iter:
                raise Exception('No convergence. Try remove start_end or reduce the difference, or increase max_iter')
    
    model.split_points_ = model.s_[mi]    
    if model.split_points_.size == 0:
        model.splitted_ = [np.array(x).flatten()]
        model.labels_ = [0]*len(x)
        return model
    else:
        model.splitted_ = get_splitted_(x, model.split_points_)
    model.labels_ = get_labels_(x, model.split_points_)
    return model

def get_split_points_(e, s):
    mi = get_mi(e)  
    return s[mi]

def get_splitted_(x, split_points_):
    splitted_ = [x[x <= split_points_[0]]]
    for idx in range(len(split_points_))[:-1]:
        splitted_.append(x[(x > split_points_[idx]) * (x <= split_points_[idx+1])])  
    splitted_.append(x[x > split_points_[-1]])
    return splitted_

def get_labels_(x, split_points_):
    labels_ = []
    for val in x:
        start_idx = 0
        label_ = get_label_(val, start_idx, split_points_)
        labels_.append(label_)
    return np.array(labels_)

def get_label_(val, idx, split_points_):
    try:
        if val < split_points_[idx]:
            return idx
        else:
            idx = get_label_(val, idx+1, split_points_)
            return idx
    except:
        return len(split_points_)

In [17]:
file_names = [
    'reaction_time_true_minmax.csv',
    'reaction_time_penalized_minmax.csv',
    'reaction_time_true_inliers_minmax.csv',
    'reaction_time_penalized_inliers_minmax.csv'
]

kde = KernelDensity(kernel='gaussian', bandwidth=0.05)
for subject_name in subject_list:
    for file_name in file_names:
        # read
        df = pd.read_csv(osp.join(path_data,subject_name,file_name))
        for idx,col_name in enumerate(df.columns):
            if 'reaction_time' in col_name:
                kde = split_kde(df[col_name].values.reshape(-1,1), model = kde, start_end=(0,1), n_groups = 3, max_iter=1000)
                df[col_name] = kde.labels_    
        df.to_csv(osp.join(path_data,subject_name,'kde_'+file_name), index=False)

## classification jenks natural breaks (to four groups, as elbow)

In [18]:
from jenkspy import jenks_breaks
class JenksNaturalBreaks:
    def __init__(self, nb_class=6):
        self.nb_class = nb_class
        
    def fit(self, x):
        self.breaks_ = jenks_breaks(x, self.nb_class)
        self.inner_breaks_ = self.breaks_[1:-1] # because inner_breaks is more 
        self.labels_ = self.predict(x)
        self.groups_ = self.group(x)
        
    def predict(self, x):
        if np.size(x) == 1:
            return np.array(self.get_label_(x, idx=0))
        else:
            labels_ = []
            for val in x:
                label_ = self.get_label_(val, idx=0)
                labels_.append(label_)
            return np.array(labels_)
        
    def group(self, x):
        x = np.array(x)
        groups_ = [x[x <= self.inner_breaks_[0]]]
        for idx in range(len(self.inner_breaks_))[:-1]:
            groups_.append(x[(x > self.inner_breaks_[idx]) * (x <= self.inner_breaks_[idx+1])])  
        groups_.append(x[x > self.inner_breaks_[-1]])
        return groups_
    
    def get_label_(self, val, idx=0):
        try:
            if val < self.inner_breaks_[idx]:
                return idx
            else:
                idx = self.get_label_(val, idx+1)
                return idx
        except:
            return len(self.inner_breaks_)

In [19]:
file_names = [
    'reaction_time_true_minmax.csv',
    'reaction_time_penalized_minmax.csv',
    'reaction_time_true_inliers_minmax.csv',
    'reaction_time_penalized_inliers_minmax.csv'
]

jnb = JenksNaturalBreaks(nb_class=4)
for subject_name in subject_list:
    for file_name in file_names:
        # read
        df = pd.read_csv(osp.join(path_data,subject_name,file_name))
        for idx,col_name in enumerate(df.columns):
            if 'reaction_time' in col_name:
                jnb.fit(df[col_name].values.reshape(-1,1))
                df[col_name] = jnb.labels_
        df.to_csv(osp.join(path_data,subject_name,'jenks_'+file_name), index=False)

c:\users\muham\appdata\local\programs\python\python37\lib\site-packages\jenkspy\core.py:98: UserWarning: Invalid values encountered (NaN or Inf) were ignored
  warnings.warn('Invalid values encountered (NaN or Inf) were ignored')


## merge data

In [20]:
path_data = 'data/individual'
path_merged = 'data/collective'

In [21]:
if not osp.exists(path_merged):
    os.makedirs(path_merged)

In [22]:
subject_list = os.listdir(path_data)

In [23]:
file_list = [
    'jenks_reaction_time_true_minmax.csv',
    'jenks_reaction_time_penalized_minmax.csv',
    'jenks_reaction_time_true_inliers_minmax.csv',
    'jenks_reaction_time_penalized_inliers_minmax.csv',
    'kde_reaction_time_penalized_inliers_minmax.csv',
    'kde_reaction_time_penalized_minmax.csv',
    'kde_reaction_time_true_inliers_minmax.csv',
    'kde_reaction_time_true_minmax.csv',
    'kmeans_reaction_time_penalized_inliers_minmax.csv',
    'kmeans_reaction_time_penalized_minmax.csv',
    'kmeans_reaction_time_true_inliers_minmax.csv',
    'kmeans_reaction_time_true_minmax.csv'
]

In [24]:
for file_name in file_list:
    df = pd.concat([pd.read_csv(osp.join(path_data,subject_name,file_name)) for subject_name in subject_list], ignore_index=True)
    df.to_csv(osp.join(path_merged,file_name), index=False)